# Data 201

Data Wrangling, S2 2021

## Assignement 2

### 20% of final grade

## Due date: 2pm Tuesday 12 October 2021

The assignment must be submitted on the DATA 201 and 422 Learn page and should be **your own pair** (or group of three if we are odd) **work**. Avoid plagiarism: if you are not sure what may constitute code plagiarism, read this http://web.science.mq.edu.au/~mtaylor/ponline/index.php?id=source-code-plagiarism. Remember, when in doubt **declare your sources**.  
As a general rule, late submission are **highly** unsuggested. The pace of the course is very quick, and you don't want to be working on this too long, as you would have less time for other assignments. I suggest you to submit what you have ready by the time the assignment is due, even if it is not everything we required. For any particular problem, please contact [Thomas Li](thomas.li@canterbury.ac.nz) or [Giulio Dalla Riva](giulio.dallariva@canterbury.ac.nz) or TAs.


## Instruction

### Total: 20 marks.

For this assignment you will work with data from the web. Your goal is to scrape it or request it, wrangle it, produce some plotting, in order to demonstrate that you are able to use the functions, plotting, and markdown format we have been learning.  
You shall hand in your work as a jupyter R notebook comprehending the code, the plots and, **very important**, enough comments and explanations for us to understand what you did, how you did it, and why you did it. Remember: better comments, means better marks.  
You can either modify this notebook or write a new one from scratch.

Do use the tidyverse syntax that's introduced in class and labs. If in doubt, refresh the lab material 1 to 7.

### Task (a): scrape it (5 marks)

Identify a website page showing information you find interesting. The information should be presented across more than one page. Select one of this pages (the rest is for later) and using `rvest` (and the suit of `tidyverse` libraries we used so far, plus any other one you may need) get the html into R.

tidyverse: collection of R packages
rvest: read webpage and expore content.

In [1]:
library(tidyverse)
library(rvest)



-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.4     v dplyr   1.0.7
v tidyr   1.1.3     v stringr 1.4.0
v readr   2.0.0     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding




In [6]:
#url_adreess_math001 <- "https://www.canterbury.ac.nz/courseinfo/GetCourseDetails.aspx?course=MATH101&occurrence=22S1(C)&year=2022"

#use readed course url to get all details we want
get_text <- function(url_readed,selector){
        text_need <- url_readed %>%
                        html_elements(selector) %>%
                        html_text()
        return(text_need)
    }
get_detail <- function(web_url){
    
    url_readed <- web_url %>% read_html()
    
    node_selector <- "#ctl00_ContentPlaceHolder1_CourseCodeLabel"
    course_code <- get_text(url_readed, node_selector)
    if (identical(course_code, character(0))){
        course_code = 'NULL'
    }

    node_selector <- "#ctl00_ContentPlaceHolder1_CourseOccurrenceTitleLabel"
    course_name <- get_text(url_readed, node_selector)
    if (identical(course_name, character(0))){
        course_name = 'NULL'
    }

    node_selector <- "#ctl00_ContentPlaceHolder1_DomesticFeeLabel"
    Domestic_fee <- get_text(url_readed, node_selector)
    if (identical(Domestic_fee, character(0))){
        Domestic_fee = 'NULL'
    }

    node_selector <- "#ctl00_ContentPlaceHolder1_InternationalFeeLabel"
    International_fee <- get_text(url_readed, node_selector)
    if (identical(International_fee, character(0))){
        International_fee = 'NULL'
    }

    node_selector <- "#ctl00_ContentPlaceHolder1_StartDateLabel"
    Start_date <- get_text(url_readed, node_selector)
    if (identical(Start_date, character(0))){
        Start_date = 'NULL'
    }

    node_selector <- "#ctl00_ContentPlaceHolder1_EndDateLabel"
    End_date <- get_text(url_readed, node_selector)
    if (identical(End_date, character(0))){
        End_date = 'NULL'
    }

    node_selector <- "#ctl00_ContentPlaceHolder1_PointsAndEftsLabel"
    Points <- get_text(url_readed, node_selector)
    if (identical(Points, character(0))){
        Points = 'NULL'
    }
    return(list(course_code, course_name, Domestic_fee, International_fee, Start_date, End_date, Points))
}


In [10]:
#url_adreess_French <- "https://www.canterbury.ac.nz/courseinfo/GetCourses.aspx?subjectnames=French&viewby=subject&year2021"             
# library(skimr)
# table_a_tage_without_na %>% skim()
# table_a_tage_without_na %>% mutate(new_col = get_detail(href))  why this can not work??????????            
                                 
one_subject_all_courses_detetal <- function(subject_url){
    read_url_subjects <- subject_url %>%
                            read_html()
    all_links <- read_url_subjects %>% 
                    html_element("#GetCourses") %>%
                    html_elements("a") 
    print(length(all_links))
    if (length(all_links) == 0 )
        {
        new_table <- data.frame("course_code"=0, "course_name"=0, "Domestic_fee"=0, "International_fee"=0, "Start_date"=0, "End_date"=0, "Points"=0, 'Webpage=0')
        return(new_table)
    }else{
    table_a_tage <- bind_rows(lapply(html_attrs(all_links), function(x) data.frame(as.list(x), stringsAsFactors=FALSE)))
    prefix <- "https://www.canterbury.ac.nz/courseinfo/"
    table_a_tage_without_na <- table_a_tage[3] %>% filter(href!="NA") %>% mutate(href = str_c(prefix, href))                                
    max_row <- nrow(table_a_tage_without_na)
    new_table <- data.frame("course_code"=0, "course_name"=0, "Domestic_fee"=0, "International_fee"=0, "Start_date"=0, "End_date"=0, "Points"=0, 'Webpage=0')

    for (i in 1:max_row){
         a <- table_a_tage_without_na[i,]
         b <- a %>% get_detail()
         new_table[nrow(new_table) + 1,] = c(b[1], b[2], b[3], b[4], b[5], b[6], b[7], a)

                                        }
    delet_first_row <- new_table[-1,] 
    return(delet_first_row) 
    }
}
#one_subject_all_courses_detetal(url_adreess_French)
#one_subject_all_courses_detetal("https://www.canterbury.ac.nz/courseinfo/GetCourseDetails.aspx?course=GISC309&year=2021")                                  

In [4]:
#https://www.canterbury.ac.nz/courseinfo/GetCourses.aspx?subjectnames=French&viewby=subject&year2021
url_adreess_subjects <- "https://www.canterbury.ac.nz/courseinfo/subjects.aspx"
get_all_course <- function(subjects_url){
                        url_subjects_a <- url_adreess_subjects %>%
                        read_html() %>% html_elements('.row') %>% html_elements('.col-md-6') %>% html_elements('a')
                        table_a_tage <- bind_rows(lapply(html_attrs(url_subjects_a), function(x) data.frame(as.list(x), stringsAsFactors=FALSE)))
                        max_row <- nrow(table_a_tage)
                        prefix_subject <- "https://www.canterbury.ac.nz/courseinfo/"

                        for(i in 1:max_row){
                             one_subject_url <- table_a_tage[i,2] %>% str_c(prefix_subject,.) %>% str_replace_all(" ", "%20")
                             a <- one_subject_all_courses_detetal(one_subject_url)
                             print(a)
                        }
                    }
get_all_course(url_adreess_subjects)  #print all courses table  then we can work on combine them

        course_code                                             course_name
2  FREN121-21S1 (C)                French Language Acquisition: Beginners A
3  FREN122-21S2 (C)                French Language Acquisition: Beginners B
4  FREN211-21S2 (C)                        Banned Books and Countercultures
5  FREN221-21S1 (C)            French Language Acquisition : Intermediate A
6  FREN222-21S2 (C)            French Language Acquisition : Intermediate B
7  FREN311-21S2 (C)                        Banned Books and Countercultures
8  FREN321-21S1 (C)                 French Language Acquisition: Advanced A
9  FREN322-21S2 (C)                 French Language Acquisition: Advanced B
10 FREN401-21S1 (C)                                         Advanced French
11 FREN418-21S2 (C)                                           Special Topic
12 FREN422-21S2 (C)                                      French Translation
13 FREN480-21S2 (C)                                          Research Essay
14  FREN650-

ERROR: Error: Can't subset columns that don't exist.
[31mx[39m Location 3 doesn't exist.
[34mi[39m There are only 0 columns.


### Task (b): wrangle it (5 marks)


Working on the page from task (a), transform the data you collected into relational data model (that is, one or more data frames). WARNING: you will need to think and motivate how you are doing this in terms of the **data model** you are trying to build; just throwing everything in a table is not enough.

In [ ]:
# your answer here

### Task (c): automate it (5 marks)

Now it's the time to scrape everything else in the website. Turn your code for tasks (a) and (b) into a function and using `purrr` (and its `map_` functions, possibly in combination with `possibly`) apply that function to all the pages you are interested in (as we have done in lab 6); the output of the task should be a relation dataset (i.e., one or more dataframes, and if more than one, explicit their relationships).

In [ ]:
# your answer here

### Task (d): plot it (2 marks)

Use graphics (via `ggplot`) to represent the data you collected in a meaningful way.

In [ ]:
# your answer here

### Task (e): APIs (2 marks)

Identify a web service offering data through an API. Access the API using `httr` and collect the information you care about. Convert the data into a dataframe (or more than one). Represent it graphically in a meaningful way.

In [ ]:
# your answer here

### Comment, explain and present the work in a clear way (1 marks)